In [2]:
import pandas as pd
from vibecheck.src.tasks import usher_parsing
from pathlib import Path

To test the accuracy of the UShER method for assigning lineages, I split the O1 Vibrio cholerae dataset 90-10 into a train and test set. Because of size differences between lineages, this split was performed on each lineage independently.

Therefore, the test set contains 10% of the UNK sequences, 10% of T1, 10% of T2, etc. For the training set, I reconstructed a phylogeny, converted to an UShER tree, and inferred the MRCA of each lineage using `matUtils annotate`. The annotated pb tree will be used as the guide tree into which I will place each sequence in the test set. Accuracy will be determined by the percentage of sequences that have a correct lineage assignment.

In [3]:
!usher -n -D -i ../vibecheck/resources/o1_cholera.no_missing.pb -v test.usher.vcf -T 12 -d usher_test/

Initializing 12 worker threads.

Loading existing mutation-annotated tree object from file ../vibecheck/resources/o1_cholera.no_missing.pb
Completed in 18 msec 

Loading VCF file
Completed in 120 msec 

Output newick files will have branch lengths equal to the number of mutations of that branch.

Found 307 missing samples.

Adding missing samples to the tree.
Current tree size (#nodes): 3855	Sample name: Africa|TUN|ERR976455|T1|1971-01-01	Parsimony score: 4	Number of parsimony-optimal placements: 2
Completed in 8 msec 

Current tree size (#nodes): 3855	Sample name: Africa|MWI|ERR976523|T1|1992-01-01	Parsimony score: 25	Number of parsimony-optimal placements: 1
Completed in 12 msec 

Current tree size (#nodes): 3855	Sample name: Africa|DZA|ERR976413|T1|1974-01-01	Parsimony score: 65	Number of parsimony-optimal placements: 1
Completed in 22 msec 

Current tree size (#nodes): 3855	Sample name: Africa|SEN|ERR976435|T1|1972-01-01	Parsimony score: 0	Number of parsimony-optimal placements: 1


Here we load the results and merge with the actual lineage calls.

In [4]:
parsed_result = usher_parsing( Path("usher_test/clades.txt"), Path( "usher_test/") )
results = pd.read_csv( parsed_result )
lin = pd.read_csv( "te_lineages.tsv", sep="\t", names=["actual", "sequence_id"]  )

results = results.merge( lin, how="left", on="sequence_id" )
results.loc[results["actual"].isna(),"actual"] = "UNDEFINED"
results["actual"] = results["actual"].replace( {"UNK" : "UNDEFINED" } )

results.head()

,sequence_id,lineage,conflict,usher_note,actual
0,Africa|TUN|ERR976455|T1|1971-01-01,T1,0.0,Usher placements: T1(2/2),T1
1,Africa|MWI|ERR976523|T1|1992-01-01,T1,0.0,Usher placements: T1(1/1),T1
2,Africa|DZA|ERR976413|T1|1974-01-01,T1,0.0,Usher placements: T1(1/1),T1
3,Africa|SEN|ERR976435|T1|1972-01-01,T1,0.0,Usher placements: T1(1/1),T1
4,Africa|CIV|ERR976425|T1|1973-01-01,T1,0.0,Usher placements: T1(1/1),T1


Now we can calculate the accuracy of UShER placement by calculating the fraction of assignments that are correct.

In [5]:
results["correct"] = results["actual"] == results["lineage"]

accuracy = results["correct"].sum() / results.shape[0]
print( f"Accuracy = {accuracy:.2%}")

Accuracy = 98.05%


In [6]:

lineage_accuracy = results.groupby("actual").apply( lambda x: x["correct"].sum() / x.shape[0] )
lineage_accuracy.name = "accuracy"
lineage_accuracy.sort_index( key = lambda x: pd.to_numeric( x.str.slice(1), errors="coerce" ) )

/var/folders/yk/ygy5y6t17g7gptxs3pqtrp240000gn/T/ipykernel_29530/3119804393.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  lineage_accuracy = results.groupby("actual").apply( lambda x: x["correct"].sum() / x.shape[0] )


actual
T1           0.916667
T2           1.000000
T5           1.000000
T6           1.000000
T7           1.000000
T8           1.000000
T9           1.000000
T10          1.000000
T11          1.000000
T12          1.000000
T13          1.000000
T15          1.000000
UNDEFINED    0.947917
Name: accuracy, dtype: float64

Accuracy of the entire test set is 98%, and the majority of these are misclassifications of the T1 lineage. This is understandable as early lineages contain very few sequences, and the boundary between T2 and T1 is ill-defined, particularly when fewer sequences are used. All other lineages have perfect recall.

In [7]:
results.loc[~results["correct"]]

,sequence_id,lineage,conflict,usher_note,actual,correct
9,Asia|BGD|GCF_000348225.2|T1|2011-01-01,T2,0.0,Usher placements: T2(1/1),T1,False
238,Asia|PAK|ERR051790|UNK|2010-01-01,T11,0.0,Usher placements: T11(1/1),UNDEFINED,False
276,Asia|PAK|ERR051745|UNK|2010-01-01,T11,0.0,Usher placements: T11(1/1),UNDEFINED,False
290,Asia|BGD|AE003852/AE003853|UNK|1975-01-01,T10,0.5,Usher placements: T10(1/2) UNDEFINED(1/2),UNDEFINED,False
305,Europe|UKR|ERR1878613|sporadic|1970-01-01,T1,0.0,Usher placements: T1(1/1),UNDEFINED,False
307,Europe|PRT|ERR976508|sporadic|1974-01-01,T1,0.0,Usher placements: T1(1/1),UNDEFINED,False


Asia|BGD sequence is at the breakpoint of T1 and T2, which is unresolved as far as I know.
The two Asia|PAK sequences are true T11 sequences but are assigned UNK because they're Asian sequences.
Second Asia|BGD sequence, no idea whats going on there. Its placed at the same location as the Zambia sequence used for the bacpage pipeline.
The two European sequences are true T1 sequences, as far as I can tell but where assigned UNK because they are European sequences.